# CustomGUI class

Written by Toshiya Sato, Matlantis Corp.(JP) <br>

## About CustomGUI

A class that calls functions stored in a .py file and executes them through a GUI.<br>
Please note that this is currently under development, and issues may occur.

In [1]:
# !pip install -U pfcc-extras  # pfcc-extras

## Preparatory Steps for Converting Functions into a GUI

Templete for the .py file："./code/utils_files/utils_templete.py"
1. Include the function for GUI implementation and the necessary import statements to execute the function in the copied .py file based on the above template.
2. If needed, add detailed settings for the function at the end of the .py file, referring to "./code/utils_files/utils_templete.py".

ex).<br>
The makesurface function is used to generate a surface.

In [6]:
# Import statement and definition of the makesurface function
from ase import Atoms
from ase.build import surface

def makesurface(
    atoms: Atoms, miller_indices=(1, 1, 1), layers=4, rep=[4, 4, 1]
) -> Atoms:
    s1 = surface(atoms, miller_indices, layers)
    s1.center(vacuum=10.0, axis=2)
    s1 = s1.repeat(rep)
    s1.set_positions(
        s1.get_positions() - [0, 0, min(s1.get_positions()[:, 2])]
    )
    s1.pbc = True
    return s1

This function can be executed as shown in the cell below. However, with CustomGUI, it can be converted into a GUI by storing the above import statements and the makesurface function in a .py file.<br>
A template has been provided in "./code/utils_files/utils_templete.py", so please review it.

Please note that the structural file used in this example was obtained from the Materials Project under the [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) license:
Rh (mp-74, Conventional Cell)

In [26]:
# Standard execution of the makesurface function
from ase.io import read
from ase.visualize import view
import ipywidgets as widgets

atoms_before = read("./input/Rh_mp-74_conventional_standard.cif")
atoms_after = makesurface(atoms_before, miller_indices=(1, 1, 1), layers=4, rep=[4, 4, 1])

v1=view(atoms_before,viewer="ngl")
v1.view.add_representation("ball+stick")
v2=view(atoms_after,viewer="ngl")
v2.view.add_representation("ball+stick")

widgets.HBox([v1.view, v2.view])

## Execution of CustomGUI

In [9]:
# import
import sys
sys.path.append("/home/jovyan/custom_gui/code/")  # Please update the path to reflect the location where the customgui.py exists
from customgui import CustomGUI

%load_ext autoreload
%autoreload 2

In [11]:
# Operation check using the .py file template. To execute the makesurface function, load the CIF file from "Read Atoms" in the left tab of the GUI and select the makesurface function. Then, choose the required arguments and click the Run button to execute.
v=CustomGUI(utils_path="./code/utils_files/utils_templete.py")
v.display()

In [4]:
atoms = v.atoms
all_atoms = v.all_atoms

print("atoms    :",atoms)
print("all_atoms:",all_atoms)

atoms    : Atoms(symbols='CH4', pbc=False, calculator=ASECalculator(...))
all_atoms: [Atoms(symbols='CH4', pbc=False, calculator=ASECalculator(...))]


### Example 1). Modeling
Here is a summary of functions available for modeling:

- smiles_to_atoms: Converts SMILES to Atoms
- generate_conformers: Generates conformers for a specified SMILES molecule (Feature introduction of pfcc-extras)
- opt_cell_size: Optimizes the cell size of bulk structures
- makesurface: Creates a surface from a bulk structure
- f_set_constraints: Fixes atoms at specified indices to prevent movement (= applies FixAtoms)
- f_add_adsorbate: Adsorbs a specified molecule (adsorbate) onto a slab at a given height and position
- f_liquid_generator: Creates a solution system (Feature introduction of pfcc-extras)
- myopt: Function for structural optimization
- AddEditor_func: Modeling tool that enables molecular movement, rotation, and addition (Feature introduction of pfcc-extras)
- get_energy_plot: Outputs a graph of the energy of Atoms

In [5]:
v=CustomGUI(read("./input/Rh_mp-74_conventional_standard.cif"),
            utils_path="./code/utils_files/utils1_modeling.py",
            added_import_functions=[("smiles_to_atoms",0),("generate_conformers",0)]
           )
v.display()

### Example 2) : Property Calculations
Here is a summary of functions available for property calculations:

- neb_calc: Executes NEB calculations by specifying the initial and final structures. As an example, input/is.json and input/fs.json are provided; assign them to IS and FS to observe the behavior.
- nvt_ast6_1: Performs NVT calculations ([Atomistic Simulation Tutorial 6-1](https://docs.matlantis.com/atomistic-simulation-tutorial/en/6_1_md-nve.html))
- show_result: Visualizes energy diagrams from NEB or MD calculation results

In [ ]:
# !pip install asap3  # use in MD calculation

In [10]:
v=CustomGUI(utils_path="./code/utils_files/utils2_property_calc.py")
v.display()

### Example 3) : Centralized Visualization Functions
Here is a summary of functions for visualization. Currently, specifying arguments for visualization functions is not supported, but this feature is under development.

- view_func: General visualization using view_ngl
- AddEditor_func: Modeling tool that enables molecular movement, rotation, and addition
- view_charge_scale: Changes atom colors based on Bader charge magnitude
- lotsatoms_view: Capable of rendering bonds for large atomic systems (up to ~5000 atoms) ([matlantis-contrib/visualize_big_atoms](https://pfcc.matlantis.com/api/docs/en/matlantis-contrib/visualize_big_atoms/lotsatomsview.html))
- get_energy_plot: Outputs a graph of Atoms energy

In [7]:
v=CustomGUI(utils_path="./code/utils_files/utils3_viewer.py")
v.display()

### Example 4) : NEB calculation
By combining modeling, property calculations, and visualization, it is possible to perform an NEB calculation using, for example, a bulk CIF file and the SMILES of an adsorbate molecule.
In this example, you need to specify arguments for the molecule, but for all other parameters, simply click the Run button without changing their values to execute the NEB calculation.

- f0_smiles_to_atoms: Specify the SMILES of the adsorbate molecule. Save the generated structure using Write Atoms in the left tab of the GUI.  
  Load ./input/Rh_mp-74_conventional_standard.cif from Read Atoms.
- f1_1_repeat_bulk: Converts the Rh bulk into a supercell.
- f2_makesurface: Creates a surface from the supercell.
- f3_set_constraints: Fixes atoms below a specified height.  
  Tip: Check the “Show constraints” box below the viewer to visualize fixed atoms.
- f4_add_adsorbate: For adsorbate, specify the molecule created by f0_smiles_to_atoms. This adsorbs the molecule onto the slab.  
  If the adsorbate dropdown is not refreshed, click the reload button on the right.
- f5_myopt: Performs structural optimization. Save as the final structure for NEB using Write Atoms.
- f6_AddEditor: Creates the initial structure. In this example, move the CO molecule away from the slab. Enter indices 64, 65 (CO molecule) in Selected atoms, then use the Editing > Move tab and click the Z+ button several times to move it about 8 Å away.
The slider next to Move determines the distance per click. For example, setting it to 2.0 Å means four clicks will move the molecule ~8 Å.  
  Note: Moving too far may cause adsorption on the opposite Rh slab due to periodic boundaries.
- f7_myopt: Optimize the initial structure. Save as the initial structure for NEB using Write Atoms.
- f8_neb_preparation: Combines the initial and final structures into a single Atoms object.
- f9_neb_calc: Executes the NEB calculation. The results will be displayed in the viewer, allowing you to observe adsorption behavior.
- f10_show_neb_result: Displays the energy diagram of the NEB calculation results.

In [9]:
v=CustomGUI(read("./input/Rh_mp-74_conventional_standard.cif"),
            utils_path="./code/utils_files/utils4_for_neb.py"
           )
v.display()